In [1]:
"""
Neuro-Snake Trainer
Based on nanoGPT by Andrej Karpathy (MIT License).
Modified for Character-Level Snake Game Physics.
"""

import os
import math
import pickle
import time
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using device: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using device: MPS (Mac Silicon)")
else:
    device = torch.device("cpu")
    print("Using device: CPU")

Using device: Tesla T4


In [3]:
# 3. The Transformer Model (nanoGPT architecture)
class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(N_EMBD, head_size, bias=False)
        self.query = nn.Linear(N_EMBD, head_size, bias=False)
        self.value = nn.Linear(N_EMBD, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))
        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(N_EMBD, N_EMBD)
        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(DROPOUT),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, N_EMBD)
        self.position_embedding_table = nn.Embedding(BLOCK_SIZE, N_EMBD)
        self.blocks = nn.Sequential(*[Block(N_EMBD, N_HEAD) for _ in range(N_LAYER)])
        self.ln_f = nn.LayerNorm(N_EMBD) # final layer norm
        self.lm_head = nn.Linear(N_EMBD, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=DEVICE)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # 1. Find the ID for the '$' character
        # We look it up in the 'stoi' dictionary you created earlier
        stop_token_id = stoi['$']

        for _ in range(max_new_tokens):
            # crop context to the last block_size tokens
            idx_cond = idx[:, -BLOCK_SIZE:]

            # get the predictions
            logits, _ = self(idx_cond)

            # focus only on the last time step and get probabilities
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)

            # sample the next token
            _, idx_next = torch.topk(probs, k=1, dim=-1)

            # --- THE MAGIC STOPPING LOGIC ---
            if idx_next.item() == stop_token_id:
                break # <--- We hit '$', so we stop immediately!
            # -------------------------------

            # append to the sequence and keep going
            idx = torch.cat((idx, idx_next), dim=1)



        return idx


In [4]:
import os
os.environ['WANDB_API_KEY'] = 'bdec11022e9685090ba22cdf9c43f1048bd67de0'

In [5]:
# --- Hyperparameters (The "Knobs") ---
# Dataset
DATA_FILE = '/content/drive/MyDrive/Colab Notebooks/snakeformer/snake_data_curriculum.txt'
BATCH_SIZE = 64        # How many independent sequences will we process in parallel?
BLOCK_SIZE = 1024      # Context length: Must be bigger than one full game turn (~600 chars)

# Model (Baby GPT)
N_EMBD = 128          # Embedding dimension
N_HEAD = 8            # Number of attention heads
N_LAYER = 4           # Number of transformer blocks
DROPOUT = 0.0         # 0.0 for faster convergence on small data

# Training
MAX_ITERS = 20000    # Total training steps
EVAL_INTERVAL = 1000    # How often to print stats
LEARNING_RATE = 1e-3   # 1e-3 or 1e-4 work well for small networks
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# -----------------------------------------------------------------------------

In [6]:
import wandb

# Initialize the run
wandb.init(
    project="snakeformer",
    name="new-dataset-20k", # Change this for different experiments
    config={
        "learning_rate": LEARNING_RATE,
        "batch_size": BATCH_SIZE,
        "block_size": BLOCK_SIZE,
        "n_embd": N_EMBD,
        "n_layer": N_LAYER,
        "max_iters": MAX_ITERS,
    }
)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: mcrimi to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Training

In [7]:

torch.manual_seed(1337)

# 1. Data Loading & Tokenizer
print(f"Loading {DATA_FILE}...")
if not os.path.exists(DATA_FILE):
    raise FileNotFoundError(f"Could not find {DATA_FILE}. Please upload it!")

with open(DATA_FILE, 'r', encoding='utf-8') as f:
    text = f.read()

# Build Vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Vocab size: {vocab_size}")
print(f"Vocabulary: {''.join(chars)}")

# Create Mappings
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Create Train/Val Split
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# 2. Data Batcher
def get_batch(split):
    data = train_data if split == 'train' else val_data
    # Randomly pick starting indices for the batch
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    # Stack them into rows
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(200) # Estimate over 200 batches
        for k in range(200):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


# 4. Training Loop
print(f"Initializing Model on {DEVICE}...")
model = GPT()
m = model.to(DEVICE)
print(f"Model parameters: {sum(p.numel() for p in m.parameters())/1e3} K")

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

print(f"Starting training for {MAX_ITERS} iterations...")
start_time = time.time()

for iter in range(MAX_ITERS):

    # --- EVALUATION BLOCK (Logs Loss + Visual Sample) ---
    if iter % EVAL_INTERVAL == 0 or iter == MAX_ITERS - 1:
        losses = estimate_loss(model)

        # 1. Generate a visual sample to see if grid is fixed
        # We use a short prompt to ask the model to complete a board
        sample_prompt = "B:\n................\n................\n.OH.............\nA:D\nT:\n"
        context = torch.tensor(encode(sample_prompt), dtype=torch.long, device=DEVICE).unsqueeze(0)
        y = model.generate(context, max_new_tokens=200)
        generated_text = decode(y[0].tolist())

        # 2. Print to console (for you)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # 3. Send to W&B
        wandb.log({
            "iter": iter,
            "train_loss": losses['train'],
            "val_loss": losses['val'],
            # We wrap it in HTML <pre> tags to keep the grid aligned in the dashboard!
            "sample_output": wandb.Html(f"<pre>{generated_text}</pre>")
        })

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    # Log the raw step loss to see the noise
    wandb.log({"step_loss": loss.item()})

print(f"Training finished in {time.time() - start_time:.2f} seconds")
wandb.finish() # Close the run

# 5. Save Artifacts (Critical for Week 3)
# Save the model weights
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/snakeformer/snake_model.pt')

# Save the tokenizer metadata
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
    'chars': chars,
    'block_size': BLOCK_SIZE,
    'n_embd': N_EMBD,
    'n_head': N_HEAD,
    'n_layer': N_LAYER
}
with open('/content/drive/MyDrive/Colab Notebooks/snakeformer/meta.pkl', 'wb') as f:
    pickle.dump(meta, f)

print("Saved 'snake_model.pt' and 'meta.pkl'. Download these for your game!")

# 6. Quick Sanity Check (Generation)
print("\n--- Generating Example Output ---")
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE) # Start with \n (id 0) or similar
print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))

Loading /content/drive/MyDrive/Colab Notebooks/snakeformer/snake_data_curriculum.txt...
Vocab size: 15
Vocabulary: 
$.:ABDFHLORTUX
Initializing Model on cuda...
Model parameters: 926.735 K
Starting training for 17000 iterations...
step 0: train loss 3.4011, val loss 3.4141
step 1000: train loss 0.3090, val loss 0.2744
step 2000: train loss 0.1887, val loss 0.1803
step 3000: train loss 0.1154, val loss 0.1253
step 4000: train loss 0.0898, val loss 0.1070
step 5000: train loss 0.0672, val loss 0.0810
step 6000: train loss 0.0585, val loss 0.0729
step 7000: train loss 0.1626, val loss 0.1867
step 8000: train loss 0.0626, val loss 0.0771
step 9000: train loss 0.0551, val loss 0.0675
step 10000: train loss 0.0521, val loss 0.0632
step 11000: train loss 0.0502, val loss 0.0602
step 12000: train loss 0.0483, val loss 0.0578
step 13000: train loss 0.0473, val loss 0.0566
step 14000: train loss 0.0463, val loss 0.0536
step 15000: train loss 0.0449, val loss 0.0528
step 16000: train loss 0.0444,

iter,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
step_loss,█▅▅▄▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▅▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
iter,16999
step_loss,0.04355
train_loss,0.04375
val_loss,0.05195


Saved 'snake_model.pt' and 'meta.pkl'. Download these for your game!

--- Generating Example Output ---

................
................
................
................
................
................
................
................
................
................
................
.............


# Load model and predict


In [19]:
import torch
import pickle
import os

# --- 1. Configuration ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# Verify these paths match where you saved them in the previous step
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/snakeformer/snake_model.pt'
META_PATH = '/content/drive/MyDrive/Colab Notebooks/snakeformer/meta.pkl'

# --- 2. Load Metadata (The "Language" and "Shape") ---
if not os.path.exists(META_PATH):
    raise FileNotFoundError(f"Meta file not found: {META_PATH}")

print(f"Loading metadata from {META_PATH}...")
with open(META_PATH, 'rb') as f:
    meta = pickle.load(f)

# Restore the Vocabulary
stoi = meta['stoi']
itos = meta['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Restore the Model Hyperparameters (CRITICAL)
# We overwrite the global variables so the GPT class uses the correct sizing
vocab_size = meta['vocab_size']
BLOCK_SIZE = meta['block_size']
N_EMBD = meta['n_embd']
N_HEAD = meta['n_head']
N_LAYER = meta['n_layer']
DROPOUT = 0.0 # No dropout during inference

print(f"Vocab Size: {vocab_size}")
print(f"Model Config: {N_LAYER} layers, {N_HEAD} heads, {N_EMBD} embedding dim")

# --- 3. Instantiate and Load the Weights ---
print("Initializing model structure...")
# The GPT class (already in your memory) will now use the globals we just set
model = GPT()

print(f"Loading weights from {MODEL_PATH}...")
# Load the state dictionary
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval() # Switch to evaluation mode (disables training-specific layers)
print("Model loaded successfully!")

# --- 4. Define the Prediction Function ---
def predict_move(prompt_str):
    # Encode
    context_idxs = encode(prompt_str)
    context = torch.tensor(context_idxs, dtype=torch.long, device=DEVICE).unsqueeze(0)

    # Generate
    # We use the generate() method you defined in your class
    output_ids = model.generate(context, max_new_tokens=1000)
    output_text = decode(output_ids[0].tolist())

    # Parse Result (Extract only the Target board)
    try:
        # We look for the part after "T:"
        if "T:" in output_text:
            generated_part = output_text.split("T:\n")[1]
            print("GENERATED TARGET (Model Output):")
            print(generated_part)
        else:
            print("RAW OUTPUT (No 'T:' found):")
            print(output_text)
    except IndexError:
        print("RAW OUTPUT:")
        print(output_text)
    print("="*40)





Loading metadata from /content/drive/MyDrive/Colab Notebooks/snakeformer/meta.pkl...
Vocab Size: 15
Model Config: 4 layers, 8 heads, 128 embedding dim
Initializing model structure...
Loading weights from /content/drive/MyDrive/Colab Notebooks/snakeformer/snake_model.pt...
Model loaded successfully!


In [28]:
# Test Case 2: Snake moving UP (Simple)
prompt_1 = """B:
................
................
................
................
................
................
................
........HF......
................
................
................
................
................
................
................
................
A:R
T:
"""


predict_move(prompt_1)

GENERATED TARGET (Model Output):
................
................
................
................
................
................
................
........OH......
................
................
................
................
................
...............F
................
................



# Fine tuning

In [15]:

# --- 1. CONFIGURATION FOR FINE-TUNING ---
# We use a lower learning rate to "polish" the model without breaking it
FINE_TUNE_LR = 1e-4      # 10x smaller than original (was 1e-3)
FINE_TUNE_ITERS = 2000   # We don't need 15k steps, just enough to learn the new cases
BATCH_SIZE = 64
BLOCK_SIZE = 1024
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Paths
OLD_MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/snakeformer/snake_model.pt'
META_PATH = '/content/drive/MyDrive/Colab Notebooks/snakeformer/meta.pkl'
NEW_DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/snakeformer/snake_data_curriculum.txt'

# --- 2. LOAD THE "OLD" VOCABULARY ---
print(f"Loading vocab from {META_PATH}...")
with open(META_PATH, 'rb') as f:
    meta = pickle.load(f)

stoi = meta['stoi']
itos = meta['itos']
vocab_size = meta['vocab_size']
print(f"Vocab size: {vocab_size}")

# --- 3. PREPARE THE NEW DATASET ---
print(f"Loading curriculum data from {NEW_DATA_PATH}...")
with open(NEW_DATA_PATH, 'r', encoding='utf-8') as f:
    text = f.read()

try:
    data = torch.tensor([stoi[c] for c in text], dtype=torch.long)
except KeyError as e:
    print(f"ERROR: Your new data contains a character '{e.args[0]}' that the old model doesn't know!")
    print("Fix: Open your curriculum generator and change that character to something common (like '$' or '.')")
    raise e

# Create Split (90% train / 10% val)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# Batch function (same as before)
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

# --- 4. INITIALIZE AND LOAD WEIGHTS ---
print("Initializing model...")
# We must use the exact global config from the meta file
model = GPT() # Uses the global class definition you already have
model.to(DEVICE)

print(f"Loading pre-trained weights from {OLD_MODEL_PATH}...")
model.load_state_dict(torch.load(OLD_MODEL_PATH, map_location=DEVICE))
print("Weights loaded. Model is ready for surgery.")

# --- 5. THE FINE-TUNING LOOP ---
optimizer = torch.optim.AdamW(model.parameters(), lr=FINE_TUNE_LR)

print(f"Starting fine-tuning for {FINE_TUNE_ITERS} steps...")
start_time = time.time()

for iter in range(FINE_TUNE_ITERS):
    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    # Logging
    if iter % 100 == 0:
        print(f"step {iter}: loss {loss.item():.4f}")

print(f"Fine-tuning finished in {time.time() - start_time:.2f} seconds")

# --- 6. SAVE THE SMART MODEL ---
# Save as a NEW file so we don't overwrite the original (just in case)
SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/snakeformer/snake_model_finetuned.pt'
torch.save(model.state_dict(), SAVE_PATH)
print(f"Saved fine-tuned model to {SAVE_PATH}")

Loading vocab from /content/drive/MyDrive/Colab Notebooks/snakeformer/meta.pkl...
Vocab size: 15
Loading curriculum data from /content/drive/MyDrive/Colab Notebooks/snakeformer/snake_data_curriculum.txt...
Initializing model...
Loading pre-trained weights from /content/drive/MyDrive/Colab Notebooks/snakeformer/snake_model.pt...
Weights loaded. Model is ready for surgery.
Starting fine-tuning for 2000 steps...
step 0: loss 0.2772
step 100: loss 0.1034
step 200: loss 0.1079
step 300: loss 0.1062
step 400: loss 0.1026
step 500: loss 0.0862
step 600: loss 0.1050
step 700: loss 0.0852
step 800: loss 0.1028
step 900: loss 0.0893
step 1000: loss 0.0974
step 1100: loss 0.0965
step 1200: loss 0.0929
step 1300: loss 0.0931
step 1400: loss 0.0862
step 1500: loss 0.0860
step 1600: loss 0.0909
step 1700: loss 0.0832
step 1800: loss 0.0825
step 1900: loss 0.0867
Fine-tuning finished in 2461.75 seconds
Saved fine-tuned model to /content/drive/MyDrive/Colab Notebooks/snakeformer/snake_model_finetuned.

In [16]:
# Verify these paths match where you saved them in the previous step
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/snakeformer/snake_model_finetuned.pt'
META_PATH = '/content/drive/MyDrive/Colab Notebooks/snakeformer/meta.pkl'

# --- 2. Load Metadata (The "Language" and "Shape") ---
if not os.path.exists(META_PATH):
    raise FileNotFoundError(f"Meta file not found: {META_PATH}")

print(f"Loading metadata from {META_PATH}...")
with open(META_PATH, 'rb') as f:
    meta = pickle.load(f)

# Restore the Vocabulary
stoi = meta['stoi']
itos = meta['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Restore the Model Hyperparameters (CRITICAL)
# We overwrite the global variables so the GPT class uses the correct sizing
vocab_size = meta['vocab_size']
BLOCK_SIZE = meta['block_size']
N_EMBD = meta['n_embd']
N_HEAD = meta['n_head']
N_LAYER = meta['n_layer']
DROPOUT = 0.0 # No dropout during inference

print(f"Vocab Size: {vocab_size}")
print(f"Model Config: {N_LAYER} layers, {N_HEAD} heads, {N_EMBD} embedding dim")

# --- 3. Instantiate and Load the Weights ---
print("Initializing model structure...")
# The GPT class (already in your memory) will now use the globals we just set
model = GPT()

print(f"Loading weights from {MODEL_PATH}...")
# Load the state dictionary
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval() # Switch to evaluation mode (disables training-specific layers)
print("Model loaded successfully!")

# --- 4. Define the Prediction Function ---
def predict_move(prompt_str):
    print("\n" + "="*40)
    print("INPUT PROMPT:")
    print(prompt_str.strip())
    print("-" * 40)

    # Encode
    context_idxs = encode(prompt_str)
    context = torch.tensor(context_idxs, dtype=torch.long, device=DEVICE).unsqueeze(0)

    # Generate
    # We use the generate() method you defined in your class
    output_ids = model.generate(context, max_new_tokens=1000)
    output_text = decode(output_ids[0].tolist())

    # Parse Result (Extract only the Target board)
    try:
        # We look for the part after "T:"
        if "T:" in output_text:
            generated_part = output_text.split("T:\n")[1]
            print("GENERATED TARGET (Model Output):")
            print(generated_part)
        else:
            print("RAW OUTPUT (No 'T:' found):")
            print(output_text)
    except IndexError:
        print("RAW OUTPUT:")
        print(output_text)
    print("="*40)

# --- 5. Run Test Cases ---

# Test Case 1: Snake moving DOWN
prompt_1 = """B:
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
.FHO............
A:L
T:
"""

# Test Case 2: Snake moving UP (Simple)
prompt_2 = """B:
................
................
................
................
................
................
................
.......O........
.......O........
.......H........
................
................
................
................
................
................
A:D
T:
"""


# Test Case 2: Snake moving UP (Simple)
prompt_3 = """B:
................
................
.F..............
................
................
................
................
................
................
................
................
................
................
................
................
.HOOOOOOO.......
A:U
T:
"""


prompt_4 = """B:
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
HOOOOOOOO.......
A:L
T:
"""

# Execute predictions
predict_move(prompt_1)
predict_move(prompt_2)
predict_move(prompt_3)
predict_move(prompt_4)

Loading metadata from /content/drive/MyDrive/Colab Notebooks/snakeformer/meta.pkl...
Vocab Size: 15
Model Config: 4 layers, 8 heads, 128 embedding dim
Initializing model structure...
Loading weights from /content/drive/MyDrive/Colab Notebooks/snakeformer/snake_model_finetuned.pt...
Model loaded successfully!

INPUT PROMPT:
B:
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
.FHO............
A:L
T:
----------------------------------------
GENERATED TARGET (Model Output):
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
.HOO............


INPUT PROMPT:
B:
................
...........